In [ ]:
import pandas as pd 
import altair as alt 
import numpy as np 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer

# Iris Dataset
Model Selection and EDA

## Summary

THese are the results

## Introduction

## Methods and Results

## Discussion

## References